In [3]:
import random
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score as auc
def adv_local_cv(df_train, df_test):    
    df_train['adv'] = 0
    df_test['adv'] = 1
    df = pd.concat([df_train, df_test],join='inner').reset_index(drop=True)
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'min_data_in_leaf': 20,
        'min_sum_hessian_in_leaf': 10.0,
        'metric': 'auc',
        'num_leaves': 2^7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'seed': 2222,
        'n_jobs': -1,
        'verbose': -1,
    }

    fold_num = 5
    seeds = [2222,42,33,51,80]
    oof_final = []
    for seed in seeds:
        params['seed']=seed
        oof = np.zeros(len(df))
        kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
        for fold, (train_idx, val_idx) in enumerate(kf.split(df, df['adv'])):
            train = lgb.Dataset(df.loc[train_idx, :],
                                df.loc[train_idx, 'adv'])
            val = lgb.Dataset(df.loc[val_idx, :],
                                df.loc[val_idx, 'adv'])
            model = lgb.train(params, train, valid_sets=[val], num_boost_round=10000, early_stopping_rounds=100)# feval=recall_score,)
            oof[val_idx] += model.predict(df.loc[val_idx, :]) / len(seeds)
            score = auc(df.loc[val_idx, 'adv'], oof[val_idx])
            oof_final.append(oof)

    return oof_final


In [ ]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score as auc
def adv_eachVar_cv(df_train, df_test, feats):
    df_train['adv'] = 0
    df_test['adv'] = 1
    df = pd.concat([df_train, df_test]).reset_index(drop=True)
    params = {
        'learning_rate': 0.1,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'seed': 2222,
        'n_jobs': 4,
        'verbose': -1,
    }

    fold_num = 5
    seeds = [2222]
    new_feats = []
    for f in feats:
        oof = np.zeros(len(df))
        for seed in seeds:
            kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
            for fold, (train_idx, val_idx) in enumerate(kf.split(df[[f]], df['adv'])):
                train = lgb.Dataset(df.loc[train_idx, [f]],
                                    df.loc[train_idx, 'adv'])
                val = lgb.Dataset(df.loc[val_idx, [f]],
                                  df.loc[val_idx, 'adv'])
                model = lgb.train(params, train, valid_sets=[val], num_boost_round=10000,  # feval=recall_score,
                                  callbacks=[lgb.early_stopping(100), lgb.log_evaluation(-1)])
                oof[val_idx] += model.predict(df.loc[val_idx, [f]]) / len(seeds)
                score = auc(df.loc[val_idx, 'adv'], oof[val_idx])
                if score > 0.85:
                    print('--------------------------------------', f, score)
                else:
                    new_feats.append(f)
                break
    return new_feats

feats = x_train.columns[1:].values
feats = get_adv_feats(df_train, df_test, feats)